In [12]:
%load_ext autoreload
%autoreload 2

In [13]:
from typing import List
from collections import defaultdict
import spacy

import en_ner_bc5cdr_md
import en_core_sci_lg
import en_ner_craft_md
from spacy.matcher import DependencyMatcher

from scispacy.abbreviation import AbbreviationDetector
from spacy.pipeline import merge_entities
from scify.nlp import show_tabs, visualise_subtrees, visualise_doc, check_for_non_trees, construct_pattern, prep_pattern

In [10]:
nlp = spacy.load("en_core_sci_md")


In [11]:
example = "causes|nsubj|START_ENTITY causes|dobj|END_ENTITY"

pattern = construct_pattern(prep_pattern(example), lemmatize=True)
sent = "The ICE inhibitor Z-YVAD-FMK causes the release of IL-1beta evoked by DNFB ."
doc = nlp(sent)

matcher = DependencyMatcher(nlp.vocab)

matcher.add("pattern1", None, pattern)
matches = matcher(doc)
print(matches, pattern)
subtree = matches[0][1][0]
visualise_subtrees(doc, subtree)

[(13439661873955722336, [[4, 3, 6]])] [{'SPEC': {'NODE_NAME': 'causes'}, 'PATTERN': {'LEMMA': 'cause'}}, {'SPEC': {'NODE_NAME': 'START_ENTITY', 'NBOR_RELOP': '>', 'NBOR_NAME': 'causes'}, 'PATTERN': {'DEP': 'nsubj', 'POS': 'NOUN'}}, {'SPEC': {'NODE_NAME': 'END_ENTITY', 'NBOR_RELOP': '>', 'NBOR_NAME': 'causes'}, 'PATTERN': {'DEP': 'dobj', 'POS': 'NOUN'}}]
Subtree:  [4, 3, 6]


In [2]:
#nlp = en_ner_bc5cdr_md.load()

abbreviation_pipe = AbbreviationDetector(nlp)
nlp.add_pipe(abbreviation_pipe)
nlp.add_pipe(merge_entities)

In [4]:
sent = open("sentences.txt", "r").read().strip().split("\n\n")
sent[:5]

['Our objective was to evaluate the potential effects of small-molecule c-Myc inhibitor , 10058-F4 , on ovarian_carcinoma cells and the underlying mechanisms by which 10058-F4 exerts its actions .',
 'The CSF levels of carbamazepine were 2.06 micrograms/mL -LRB- ie , 31 % of plasma levels , which equaled 6.55 micrograms/mL -RRB- ; CSF -10,11-epoxide concentrations averaged 0.91 micrograms/mL in 18 subjects -LRB- 63 % of those found in plasma -RRB- .',
 'Mutational analysis of p59fyn revealed that a 10-amino-acid sequence in the unique amino-terminal domain of p59fyn was responsible for the association with zeta .',
 'The cDNA encodes S1P , an intraluminal 1052-amino-acid membrane-bound subtilisin-like protease .',
 "Alzheimer's disease is a late-stage form of dementia."]

# TODO
0) get a testset!
1) Take all entity pairs [known entities only]
2) filter by (haveVerbBetweenThem)
2.5) what to do if 2? "She huffed and puffed on her soup" 
2.8) handle activa-passiva .flip()
3) Get training set (BioCause)
4) Measure Precision


In [6]:
chunks(doc), [(token.pos_, token.tag_, spacy.explain(token.tag_)) for token in doc]

([TREM2, its association, the involvement, as a consequence],
 [('NOUN', 'NN', 'noun, singular or mass'),
  ('AUX', 'VBZ', 'verb, 3rd person singular present'),
  ('DET', 'DT', 'determiner'),
  ('NOUN', 'NN', 'noun, singular or mass'),
  ('VERB', 'VBN', 'verb, past participle'),
  ('ADP', 'IN', 'conjunction, subordinating or preposition'),
  ('NOUN', 'NNS', 'noun, plural'),
  ('ADP', 'IN', 'conjunction, subordinating or preposition'),
  ('DET', 'DT', 'determiner'),
  ('NOUN', 'NN', 'noun, singular or mass'),
  ('PUNCT', ',', 'punctuation mark, comma'),
  ('CCONJ', 'CC', 'conjunction, coordinating'),
  ('PRON', 'PRP$', 'pronoun, possessive'),
  ('NOUN', 'NN', 'noun, singular or mass'),
  ('ADP', 'IN', 'conjunction, subordinating or preposition'),
  ('NOUN', 'NN', 'noun, singular or mass'),
  ('VERB', 'VBZ', 'verb, 3rd person singular present'),
  ('DET', 'DT', 'determiner'),
  ('NOUN', 'NN', 'noun, singular or mass'),
  ('ADP', 'IN', 'conjunction, subordinating or preposition'),
  ('ADJ

In [9]:
from spacy.matcher import Matcher


# Iterate over the matches
for match_id, start, end in matches:
    # Get the matched span
    matched_span = doc2[start:end]
    print(matched_span.text)

love loving


In [24]:
#25682284	3	17-AAG	446,452	HSP90	500,505	17-AAG	HSP90	MESH:C112765	104434(Tax:10090)	
#Chemical	Gene	inhibited|nsubj|START_ENTITY inhibited|dobj|expression expression|nmod|END_ENTITY	
biomed = "Furthermore , 17-AAG also significantly inhibited the expression of HSP90 and its client proteins ."


In [25]:
visualise_doc(doc4)
visualise_doc(nlp(biomed))

/Users/markus/opt/anaconda3/envs/markus_nlp/lib/python3.8/site-packages/spacy/displacy/__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [12]:
show_tree(doc)

{'TREM2': [],
 'is': [],
 'a': [],
 'transmembrane receptor': [TREM2, is, a, expressed, ,, and, association],
 'expressed': [cells],
 'in': [],
 'cells': [in, myeloid lineage],
 'of': [],
 'the': [],
 'myeloid lineage': [of, the],
 ',': [],
 'and': [],
 'its': [],
 'association': [its, Alzheimer's disease],
 'with': [],
 "Alzheimer's disease": [with],
 'supports': [transmembrane receptor, involvement, .],
 'involvement': [the, inflammatory pathways, cause, ,, rather, consequence],
 'immune': [and],
 'inflammatory pathways': [of, immune],
 'cause': [in, the, disease],
 'disease': [of, the],
 'rather': [than],
 'than': [],
 'as': [],
 'consequence': [as, a, disease],
 '.': []}

In [16]:
doc[1].ent_iob
#IOB code of named entity tag. 3 means the token begins an entity, 2 means it is outside an entity, 1 means 
#it is inside an entity, and 0 means no entity tag is set.

2

In [18]:
from scify.networks import get_edges, triples_from_pairs, pair_down

G = nx.Graph(get_edges(doc))
all_pairs = [*combinations([token.i for token in doc if token.ent_iob !=2], 2)] #generators suck bc state is weird and fleeting

pairs = pair_down(all_pairs)

triples = triples_from_pairs(pairs, G, doc)
#nx.shortest_path(G, doc[1], doc[2]), 

triples

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


[(expressed, TREM2, expressed),
 (expressed, TREM2, cells),
 (expressed, TREM2, myeloid lineage),
 (supports, TREM2, involvement),
 (supports, TREM2, immune),
 (supports, TREM2, inflammatory pathways),
 (supports, TREM2, disease),
 (supports, TREM2, disease),
 (expressed, transmembrane receptor, expressed),
 (expressed, transmembrane receptor, cells),
 (expressed, transmembrane receptor, myeloid lineage),
 (supports, transmembrane receptor, involvement),
 (supports, transmembrane receptor, immune),
 (supports, transmembrane receptor, inflammatory pathways),
 (supports, transmembrane receptor, disease),
 (supports, transmembrane receptor, disease),
 (expressed, expressed, cells),
 (expressed, expressed, myeloid lineage),
 (expressed, expressed, association),
 (expressed, expressed, Alzheimer's disease),
 (expressed, expressed, involvement),
 (expressed, expressed, immune),
 (expressed, expressed, inflammatory pathways),
 (expressed, expressed, disease),
 (expressed, expressed, disease),

In [19]:
doc

TREM2 is a transmembrane receptor expressed in cells of the myeloid lineage, and its association with Alzheimer's disease supports the involvement of immune and inflammatory pathways in the cause of the disease, rather than as a consequence of the disease.

In [20]:
visualise_doc(doc, compact=True)
head, *middle, tail = [1, 2, 3, 4, 5]
chunks(doc)

[TREM2, its association, the involvement, as a consequence]

In [21]:
doc1 = doc
visualise_doc(doc1)
pat1 = [{'SPEC': {'NODE_NAME': 'node1'}, 'PATTERN': {'TEXT': {"LEMMA": "cause"}}},
 {'SPEC': {'NODE_NAME': 'START_ENTITY',
   'NBOR_RELOP': '>',
   'NBOR_NAME': 'node1'},
  'PATTERN': {'DEP': 'nsubj', 'ENT_TYPE': {'NOT_IN': ['']}}}
      , {"SPEC": {"NODE_NAME": 'END_ENTITY', "NBOR_RELOP": ">", "NBOR_NAME":"node1"},
        "PATTERN": {"DEP": "dobj"}
        } 
]

print(chunks(doc1))
matcher = DependencyMatcher(nlp.vocab)
matcher.add("pattern1", None, pat1)
match = matcher(doc1)[0]
match
subtree = match[1][0]
visualise_subtrees(doc1, subtree)

[TREM2, its association, the involvement, as a consequence]
Subtree:  [16, 3, 18]
